In [1]:
import pandas as pd
from pyvi import ViUtils

In [2]:
tgdd_df = pd.read_csv('../data/tgdd.csv')
for column in tgdd_df:
    print(column)

TÊN SẢN PHẨM 
GIÁ MỚI 
GIÁ CŨ 
CÔNG NGHỆ MÀN HÌNH
ĐỘ PHÂN GIẢI
MÀN HÌNH RỘNG
ĐỘ SÁNG TỐI ĐA
MẶT KÍNH CẢM ỨNG
CAMERA SAU
CAMERA TRƯỚC
HỆ ĐIỀU HÀNH
CPU
TỐC ĐỘ CPU
GPU 
RAM
BỘ NHỚ TRONG 
THẺ NHỚ
SIM
BLUETOOTH
CỔNG KẾT NỐI 
DUNG LƯỢNG PIN 
KÍCH THƯỚC - KHỐI LƯỢNG
THỜI ĐIỂM RA MẮT


In [3]:
drop_columns = ['GIÁ CŨ ', 'ĐỘ PHÂN GIẢI', 'ĐỘ SÁNG TỐI ĐA', 'MẶT KÍNH CẢM ỨNG', 'TỐC ĐỘ CPU', 'GPU ', 'THẺ NHỚ', 'CỔNG KẾT NỐI ', 'THỜI ĐIỂM RA MẮT', 'KÍCH THƯỚC - KHỐI LƯỢNG', 'CAMERA SAU', 'CAMERA TRƯỚC', 'HỆ ĐIỀU HÀNH', 'BLUETOOTH']

for column in drop_columns:
    del tgdd_df[column]

tgdd_df.head()

,TÊN SẢN PHẨM,GIÁ MỚI,CÔNG NGHỆ MÀN HÌNH,MÀN HÌNH RỘNG,CPU,RAM,BỘ NHỚ TRONG,SIM,DUNG LƯỢNG PIN
0,Điện thoại Samsung Galaxy Z Fold3 5G 256GB,35.990.000₫,Dynamic AMOLED 2X,"Chính 7.6"" & Phụ 6.2"" - Tần số quét 120 Hz",Snapdragon 888 8 nhân,12 GB,256 GB,2 Nano SIM + 1 eSIM,4400 mAh
1,Điện thoại iPhone 12 Pro Max 512GB,NaN,OLED,"6.7"" - Tần số quét 60 Hz",Apple A14 Bionic 6 nhân,6 GB,512 GB,1 Nano SIM & 1 eSIM,3687 mAh
2,Điện thoại Samsung Galaxy S22 Ultra 5G 128GB,30.990.000₫,Dynamic AMOLED 2X,"6.8"" - Tần số quét 120 Hz",Snapdragon 8 Gen 1 8 nhân,8 GB,128 GB,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,5000 mAh
3,Điện thoại OPPO Find X5 Pro 5G,NaN,AMOLED,"6.7"" - Tần số quét 120 Hz",Snapdragon 8 Gen 1 8 nhân,12 GB,256 GB,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,5000 mAh
4,Điện thoại iPhone 13 Pro Max 128GB,NaN,OLED,"6.7"" - Tần số quét 120 Hz",Apple A15 Bionic 6 nhân,6 GB,128 GB,1 Nano SIM & 1 eSIM,4352 mAh


# Xử lý tên cột

In [4]:
for column in tgdd_df.columns:
    tgdd_df = tgdd_df.rename(columns={column: ViUtils.remove_accents(column).decode("utf-8").lower().strip().replace(' ', '_')})

# Xử lý trường tên

In [5]:
tgdd_df['ten_san_pham'] = tgdd_df['ten_san_pham'].apply(lambda s: s.replace('Điện thoại ', '').strip())
tgdd_df = tgdd_df.rename(columns={'ten_san_pham': 'ten'})

In [6]:
tgdd_df = tgdd_df[tgdd_df['gia_moi'].notna()]
tgdd_df['gia_moi'] = tgdd_df['gia_moi'].apply(lambda s: str(s).replace('₫', '').replace('.', '').replace('*', '').strip())
tgdd_df['gia_moi'] = tgdd_df['gia_moi'].apply(lambda s: int(s))

In [7]:
def process_screen_size(s):
    size = s.split(' - ')[0]
    if "&" in s:
        size = size.split(' ')
        size = float(size[1][:-1]) + float(size[4][:-1])
    else:
        size = float(size[:-1])
    return size
tgdd_df['kich_thuoc_man_hinh'] = tgdd_df['man_hinh_rong'].apply(process_screen_size)

# def process_screen_frequency(s):
#     try:
#         frequency = s.split(' - ')[1]
#         frequency = int(frequency.split(' ')[3])
#     except:
#         frequency = 0
#     return frequency

# tgdd_df['tan_so'] = tgdd_df['man_hinh_rong'].apply(process_screen_frequency)
tgdd_df = tgdd_df.drop(['man_hinh_rong'], axis=1)

In [8]:
def process_ram(s):
    try:
        return int(s[:-3])
    except:
        return 0

tgdd_df['ram'] = tgdd_df['ram'].apply(process_ram)

In [9]:
def process_rom(s):
    try:
        return float(s[:-3])
    except:
        return 0

tgdd_df['bo_nho_trong'] = tgdd_df['bo_nho_trong'].apply(process_rom)

In [10]:
tgdd_df = tgdd_df.rename(columns={'dung_luong_pin': 'pin'})

In [11]:
tgdd_df.head()

,ten,gia_moi,cong_nghe_man_hinh,cpu,ram,bo_nho_trong,sim,pin,kich_thuoc_man_hinh
0,Samsung Galaxy Z Fold3 5G 256GB,35990000,Dynamic AMOLED 2X,Snapdragon 888 8 nhân,12,256.0,2 Nano SIM + 1 eSIM,4400 mAh,13.8
2,Samsung Galaxy S22 Ultra 5G 128GB,30990000,Dynamic AMOLED 2X,Snapdragon 8 Gen 1 8 nhân,8,128.0,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,5000 mAh,6.8
6,iPhone 12 Pro 256GB,26290000,OLED,Apple A14 Bionic 6 nhân,6,256.0,1 Nano SIM & 1 eSIM,2815 mAh,6.1
7,Samsung Galaxy S22+ 5G 128GB,25990000,Dynamic AMOLED 2X,Snapdragon 8 Gen 1 8 nhân,8,128.0,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,4500 mAh,6.6
9,Samsung Galaxy S22 5G 128GB,21990000,Dynamic AMOLED 2X,Snapdragon 8 Gen 1 8 nhân,8,128.0,2 Nano SIM hoặc 1 Nano SIM + 1 eSIM,3700 mAh,6.1


In [12]:
tgdd_df.to_csv('../clean_data/tgdd.csv', index=False)